# Workflow as Agent with Human-in-the-Loop

## Overview

This notebook demonstrates how to wrap a workflow as an agent and implement human escalation patterns. The workflow creates a **Worker-Reviewer** cycle where uncertain decisions are escalated to a human manager for approval.

### Key Concepts:

1. **WorkflowAgent**: Wraps a workflow to expose an agent-like interface
2. **Human Escalation**: When the reviewer lacks confidence, it escalates to a human
3. **Bidirectional Flow**: Worker ↔ Reviewer ↔ RequestInfoExecutor
4. **Function Call Protocol**: Human feedback is delivered via `FunctionCallContent` and `FunctionResultContent`

### Workflow Pattern:

```
Worker (generates response)
    ↓
Reviewer (evaluates quality)
    ↓ (if uncertain)
RequestInfoExecutor (human manager)
    ↓
Reviewer (receives human decision)
    ↓
Worker (incorporates feedback if needed)
```

### When to Use This Pattern:

- Critical decisions requiring human oversight
- High-stakes outputs (legal, medical, financial)
- Quality assurance workflows
- Compliance and approval gates

## Prerequisites

- OpenAI API key configured (environment variable `OPENAI_API_KEY`)
- Agent Framework installed: `pip install agent-framework`
- Understanding of `WorkflowBuilder`, `Executor`, and message handlers

## Setup and Imports

In [ ]:
import asyncio
import sys
from collections.abc import Mapping
from dataclasses import dataclass
from pathlib import Path
from typing import Any, cast

import os
from dotenv import load_dotenv
from agent_framework import (
    ChatMessage,
    Executor,
    FunctionCallContent,
    FunctionResultContent,
    RequestInfoExecutor,
    RequestInfoMessage,
    RequestResponse,
    Role,
    WorkflowAgent,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.openai import OpenAIChatClient
# Load environment variables from .env file
load_dotenv('../../.env')


## Import Worker and Review Message Types

We'll reuse the `Worker`, `ReviewRequest`, and `ReviewResponse` classes from the reflection pattern sample. In a real scenario, these would be defined in shared modules.

In [ ]:
# For this notebook, we'll define simplified versions inline
# In production, import from: getting_started.workflows.agents.workflow_as_agent_reflection_pattern

from uuid import uuid4
from agent_framework import AgentRunResponseUpdate, AgentRunUpdateEvent, ChatClientProtocol, Contents

@dataclass
class ReviewRequest:
    """Structured request passed from Worker to Reviewer for evaluation."""
    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]

@dataclass
class ReviewResponse:
    """Structured response from Reviewer back to Worker."""
    request_id: str
    feedback: str
    approved: bool

class Worker(Executor):
    """Executor that generates responses and incorporates feedback when necessary."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]) -> None:
        print("Worker: Received user messages, generating response...")

        # Initialize chat with system prompt.
        messages = [ChatMessage(role=Role.SYSTEM, text="You are a helpful assistant.")]
        messages.extend(user_messages)

        print("Worker: Calling LLM to generate response...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: Response generated: {response.messages[-1].text}")

        # Add agent messages to context.
        messages.extend(response.messages)

        # Create review request and send to Reviewer.
        request = ReviewRequest(request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages)
        print(f"Worker: Sending response for review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        # Track request for possible retry.
        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]) -> None:
        print(f"Worker: Received review for request {review.request_id[:8]} - Approved: {review.approved}")

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID in review: {review.request_id}")

        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("Worker: Response approved. Emitting to external consumer...")
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit approved result to external consumer via AgentRunUpdateEvent.
            await ctx.add_event(
                AgentRunUpdateEvent(self.id, data=AgentRunResponseUpdate(contents=contents, role=Role.ASSISTANT))
            )
            return

        print(f"Worker: Response not approved. Feedback: {review.feedback}")
        print("Worker: Regenerating response with feedback...")

        # Incorporate review feedback.
        messages.append(ChatMessage(role=Role.SYSTEM, text=review.feedback))
        messages.append(
            ChatMessage(role=Role.SYSTEM, text="Please incorporate the feedback and regenerate the response.")
        )
        messages.extend(request.user_messages)

        # Retry with updated prompt.
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: New response generated: {response.messages[-1].text}")

        messages.extend(response.messages)

        # Send updated request for re-review.
        new_request = ReviewRequest(
            request_id=review.request_id, user_messages=request.user_messages, agent_messages=response.messages
        )
        await ctx.send_message(new_request)

        # Track new request for further evaluation.
        self._pending_requests[new_request.request_id] = (new_request, messages)

## Define Human Review Request

This message type wraps the review request for escalation to a human manager.

In [ ]:
@dataclass
class HumanReviewRequest(RequestInfoMessage):
    """A request message type for escalation to a human reviewer."""
    agent_request: ReviewRequest | None = None

## Create Reviewer with Human Escalation

Unlike the automated reviewer in the reflection pattern, this reviewer **always escalates** to a human manager. In production, you'd add logic to escalate only when confidence is low.

### Handler Methods:

#### 1. `review`
- Receives `ReviewRequest` from Worker
- Decides to escalate (in this sample, always)
- Sends `HumanReviewRequest` to `RequestInfoExecutor`

#### 2. `accept_human_review`
- Receives `RequestResponse[HumanReviewRequest, ReviewResponse]` with human decision
- Extracts the `ReviewResponse` from human feedback
- Forwards decision back to Worker

In [ ]:
class ReviewerWithHumanInTheLoop(Executor):
    """Executor that always escalates reviews to a human manager."""

    def __init__(self, worker_id: str, request_info_id: str, reviewer_id: str | None = None) -> None:
        unique_id = reviewer_id or f"{worker_id}-reviewer"
        super().__init__(id=unique_id)
        self._worker_id = worker_id
        self._request_info_id = request_info_id

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext[ReviewResponse | HumanReviewRequest]) -> None:
        # In this simplified example, we always escalate to a human manager.
        # See workflow_as_agent_reflection.py for an implementation
        # using an automated agent to make the review decision.
        print(f"Reviewer: Evaluating response for request {request.request_id[:8]}...")
        print("Reviewer: Escalating to human manager...")

        # Forward the request to a human manager by sending a HumanReviewRequest.
        await ctx.send_message(
            HumanReviewRequest(agent_request=request),
            target_id=self._request_info_id,
        )

    @handler
    async def accept_human_review(
        self, response: RequestResponse[HumanReviewRequest, ReviewResponse], ctx: WorkflowContext[ReviewResponse]
    ) -> None:
        # Accept the human review response and forward it back to the Worker.
        human_response = response.data
        assert isinstance(human_response, ReviewResponse)
        print(f"Reviewer: Accepting human review for request {human_response.request_id[:8]}...")
        print(f"Reviewer: Human feedback: {human_response.feedback}")
        print(f"Reviewer: Human approved: {human_response.approved}")
        print("Reviewer: Forwarding human review back to worker...")
        await ctx.send_message(human_response, target_id=self._worker_id)

## Build Workflow and Wrap as Agent

### Workflow Architecture:

1. **Worker ↔ Reviewer**: Bidirectional edge for review cycles
2. **Reviewer → RequestInfoExecutor**: Escalation path for human input
3. **RequestInfoExecutor → Reviewer**: Human response return path
4. **`.as_agent()`**: Wraps the workflow with agent-like interface

### Key Benefits of WorkflowAgent:

- Exposes `run()` and `run_stream()` methods like any agent
- Handles `FunctionCallContent` for `REQUEST_INFO_FUNCTION_NAME` internally
- Allows workflows to be composed as building blocks in larger systems

In [ ]:
async def run_workflow_with_human_escalation() -> None:
    print("Starting Workflow Agent with Human-in-the-Loop Demo")
    print("=" * 50)

    # Create executors for the workflow.
    print("Creating chat client and executors...")
    mini_chat_client = OpenAIChatClient(model_id="gpt-4o-mini")
    worker = Worker(id="sub-worker", chat_client=mini_chat_client)
    request_info_executor = RequestInfoExecutor(id="request_info")
    reviewer = ReviewerWithHumanInTheLoop(worker_id=worker.id, request_info_id=request_info_executor.id)

    print("Building workflow with Worker ↔ Reviewer cycle...")
    # Build a workflow with bidirectional communication between Worker and Reviewer,
    # and escalation paths for human review.
    agent = (
        WorkflowBuilder()
        .add_edge(worker, reviewer)  # Worker sends requests to Reviewer
        .add_edge(reviewer, worker)  # Reviewer sends feedback to Worker
        .add_edge(reviewer, request_info_executor)  # Reviewer requests human input
        .add_edge(request_info_executor, reviewer)  # Human input forwarded back to Reviewer
        .set_start_executor(worker)
        .build()
        .as_agent()  # Convert workflow into an agent interface
    )

    print("Running workflow agent with user query...")
    print("Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'")
    print("-" * 50)

    # Run the agent with an initial query.
    response = await agent.run(
        "Write code for parallel reading 1 million Files on disk and write to a sorted output file."
    )

    # Locate the human review function call in the response messages.
    human_review_function_call: FunctionCallContent | None = None
    for message in response.messages:
        for content in message.contents:
            if isinstance(content, FunctionCallContent) and content.name == WorkflowAgent.REQUEST_INFO_FUNCTION_NAME:
                human_review_function_call = content

    # Handle the human review if required.
    if human_review_function_call:
        # Parse the human review request arguments.
        human_request_args = human_review_function_call.arguments
        if isinstance(human_request_args, str):
            request: WorkflowAgent.RequestInfoFunctionArgs = WorkflowAgent.RequestInfoFunctionArgs.from_json(
                human_request_args
            )
        elif isinstance(human_request_args, Mapping):
            request = WorkflowAgent.RequestInfoFunctionArgs.from_dict(dict(human_request_args))
        else:
            raise TypeError("Unexpected argument type for human review function call.")

        request_payload_obj: Any = request.data
        if not isinstance(request_payload_obj, Mapping):
            raise ValueError("Human review request payload must be a mapping.")
        request_payload = cast(Mapping[str, Any], request_payload_obj)

        agent_request_obj = request_payload.get("agent_request")
        if not isinstance(agent_request_obj, Mapping):
            raise ValueError("Human review request must include agent_request mapping data.")
        agent_request_data = cast(Mapping[str, Any], agent_request_obj)

        request_id_obj = agent_request_data.get("request_id")
        if not isinstance(request_id_obj, str):
            raise ValueError("Human review request_id must be a string.")
        request_id_value = request_id_obj

        # Mock a human response approval for demonstration purposes.
        human_response = ReviewResponse(request_id=request_id_value, feedback="Approved", approved=True)

        # Create the function call result object to send back to the agent.
        human_review_function_result = FunctionResultContent(
            call_id=human_review_function_call.call_id,
            result=human_response,
        )
        # Send the human review result back to the agent.
        response = await agent.run(ChatMessage(role=Role.TOOL, contents=[human_review_function_result]))
        print(f"📤 Agent Response: {response.messages[-1].text}")

    print("=" * 50)
    print("Workflow completed!")

## Run the Workflow

In [ ]:
await run_workflow_with_human_escalation()

## Expected Output

```
Starting Workflow Agent with Human-in-the-Loop Demo
==================================================
Creating chat client and executors...
Building workflow with Worker ↔ Reviewer cycle...
Running workflow agent with user query...
Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'
--------------------------------------------------
Worker: Received user messages, generating response...
Worker: Calling LLM to generate response...
Worker: Response generated: Here's a Python implementation using concurrent.futures...
Worker: Sending response for review (ID: 12345678)
Reviewer: Evaluating response for request 12345678...
Reviewer: Escalating to human manager...
Reviewer: Accepting human review for request 12345678...
Reviewer: Human feedback: Approved
Reviewer: Human approved: True
Reviewer: Forwarding human review back to worker...
Worker: Received review for request 12345678 - Approved: True
Worker: Response approved. Emitting to external consumer...
📤 Agent Response: Here's a Python implementation using concurrent.futures...
==================================================
Workflow completed!
```

## Key Takeaways

### 1. WorkflowAgent Wrapper
- Use `.as_agent()` to convert workflows into agent-like interfaces
- Enables composition of complex workflows as reusable building blocks
- Exposes standard `run()` and `run_stream()` methods

### 2. Human Escalation Pattern
- `RequestInfoExecutor` provides built-in support for external input
- WorkflowAgent exposes `REQUEST_INFO_FUNCTION_NAME` as a function call
- Human responses delivered via `FunctionResultContent`

### 3. Bidirectional Communication
- Worker ↔ Reviewer edges enable iterative refinement
- Reviewer can escalate to human or send feedback directly to Worker
- Worker regenerates response based on feedback until approved

### 4. Message Type Design
- `HumanReviewRequest` extends `RequestInfoMessage` for escalation
- `ReviewResponse` carries approval decision and feedback
- Strong typing ensures correct handler matching

### 5. Production Considerations
- Add confidence scoring to determine when escalation is needed
- Implement timeout/retry logic for human responses
- Log escalation events for audit trails
- Consider async human response handling (webhook callbacks)

### 6. When to Use This vs. Automated Review
- **Human-in-the-Loop**: Critical decisions, compliance gates, low confidence scenarios
- **Automated Review**: High-throughput, well-defined criteria, sufficient training data